# Introduction
## This Project aims to analize different eXplanable AI techniques. This project can be divided in 3 macro-section.
1) In the first macro-section, different XAI techniques (i.e., RISE, LIME, Grad-CAM, Grad-CAM++) are applied to an image classification task, followed by the extraction of saliency maps using these various methods.
2) In the other section, these techniques are compared using different metrics (Insertion, Deletion, Pointing Game), analyzing their respective advantages and disadvantages.
3) In the last section, an XAI technique is applied in the image classification domain to investigate the presence of biases in the models.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms, utils
from torchvision.models import ResNet50_Weights
from torch.nn.functional import conv2d
import torch.utils.data
from torch.utils.data import DataLoader, Subset, Dataset
import torch.nn.functional as F

import os
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *

import cv2

from utils import *
from evaluation import CausalMetric, auc, gkern
from explanation import RISE, RISEBatch

import PIL

from lime import lime_image
from skimage.segmentation import mark_boundaries

cudnn.benchmark = True

device = 'cuda' if torch.cuda.is_available else 'cpu'
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
args = Dummy()


args.workers = 4

args.datadir = '/kaggle/input/imagenet100/val.X'

# extract a subset of range (100) images to apply the classification model and the eXplainability techniques
args.range = range(0, 100)

# Size of imput images
args.input_size = (224, 224)

args.gpu_batch = 10
args.batch_size = 10

dataset = datasets.ImageFolder(args.datadir, preprocess)

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True, sampler=RangeSampler(args.range))

print('Dataset size: {:}'.format(len(dataset)))
print('number of classes: {:}'.format(len(dataset.classes)))
print('Selected dataloader size:{: }'.format(len(data_loader) * data_loader.batch_size))

## Resnet50 pretrained on the training set of the imageNet dataset
## In our case, it is necessary to define a neural network output layer with 100 units, because we are working on a subset of ImageNet that contains only 100 out of 1000 classes.

In [ ]:
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False

# SECTION 1: XAI Overview and comparison

## 1) Simple example of a saliency map extracted using the RISE eXplainability technique
### Note: some of the classes and utility functions are taken from the RISE paper (https://arxiv.org/pdf/1806.07421) and its public code on GitHub (https://github.com/eclique/RISE).

In [ ]:
explainer = RISE(model, args.input_size, args.gpu_batch)

maskspath = 'masks.npy'
generate_new = True

if generate_new or not os.path.isfile(maskspath):
    explainer.generate_masks(N=1000, s=8, p1=0.1, savepath=maskspath)
else:
    explainer.load_masks(maskspath)
    print('Masks are loaded.')

### This method is extracted from the RISE paper and return a saliency map of the top_k classes (selected 5) predicted of a particular image (default value is 3)

In [ ]:
def example(img, top_k=3):
    saliency = explainer(img.cuda()).cpu().numpy()
    #Model restituisce solo le probabilità softmax
    p, c = torch.topk(model(img.cuda()), k=top_k)
    p, c = p[0], c[0]
    
    plt.figure(figsize=(10, 5*top_k))
    for k in range(top_k):
        plt.subplot(top_k, 2, 2*k+1)
        plt.axis('off')
        plt.title('{:.2f}% {}'.format(100*p[k], get_class_name(c[k])))
        tensor_imshow(img[0])

        plt.subplot(top_k, 2, 2*k+2)
        plt.axis('off')
        plt.title(get_class_name(c[k]))
        tensor_imshow(img[0])
        sal = saliency[c[k]]
        plt.imshow(sal, cmap='jet', alpha=0.5)
        plt.colorbar(fraction=0.046, pad=0.04)

    plt.show()

In [ ]:
example(read_tensor('/kaggle/input/imagenet100/val.X/n01443537/ILSVRC2012_val_00000994.JPEG'), 5)

## 2) Comparision between different eXplainability techniques:
 1. RISE
 2. LIME
 3. GradCAM
 4. GradCAM++

### 2.0.1) Dataset subset extraction

In [ ]:
# Dataset creation using Imagefolder
dataset = datasets.ImageFolder(args.datadir)

# Select a subset using the specified indexes
subset_indices = list(args.range)
subset = Subset(dataset, subset_indices)

### 2.0.2) GradCAM & GradCAM ++ implementations

In [ ]:
def find_resnet_layer(arch, target_layer_name):
    if 'layer' in target_layer_name:
        hierarchy = target_layer_name.split('_')
        layer_num = int(hierarchy[0].lstrip('layer'))
        if layer_num == 1:
            target_layer = arch.layer1
        elif layer_num == 2:
            target_layer = arch.layer2
        elif layer_num == 3:
            target_layer = arch.layer3
        elif layer_num == 4:
            target_layer = arch.layer4
        else:
            raise ValueError('unknown layer : {}'.format(target_layer_name))

        if len(hierarchy) >= 2:
            bottleneck_num = int(hierarchy[1].lower().lstrip('bottleneck').lstrip('basicblock'))
            target_layer = target_layer[bottleneck_num]

        if len(hierarchy) >= 3:
            target_layer = target_layer._modules[hierarchy[2]]
                
        if len(hierarchy) == 4:
            target_layer = target_layer._modules[hierarchy[3]]

    else:
        target_layer = arch._modules[target_layer_name]

    return target_layer

def visualize_cam(mask, img):
    """Make heatmap from mask and synthesize GradCAM result image using heatmap and img.
    Args:
        mask (torch.tensor): mask shape of (1, 1, H, W) and each element has value in range [0, 1]
        img (torch.tensor): img shape of (1, 3, H, W) and each pixel value is in range [0, 1]
        
    Return:
        heatmap (torch.tensor): heatmap img shape of (3, H, W)
        result (torch.tensor): synthesized GradCAM result of same shape with heatmap.
    """
    
    heatmap = cv2.applyColorMap(np.uint8(255 * mask.squeeze()), cv2.COLORMAP_JET)
    heatmap = torch.from_numpy(heatmap).permute(2, 0, 1).float().div(255)
    b, g, r = heatmap.split(1)
    heatmap = torch.cat([r, g, b])
    
    result = heatmap+img.cpu()
    result = result.div(result.max()).squeeze()
    
    return heatmap, result

def denormalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.mul(std).add(mean)


def normalize(tensor, mean, std):
    if not tensor.ndimension() == 4:
        raise TypeError('tensor should be 4D')

    mean = torch.FloatTensor(mean).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)
    std = torch.FloatTensor(std).view(1, 3, 1, 1).expand_as(tensor).to(tensor.device)

    return tensor.sub(mean).div(std)


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return self.do(tensor)
    
    def do(self, tensor):
        return normalize(tensor, self.mean, self.std)
    
    def undo(self, tensor):
        return denormalize(tensor, self.mean, self.std)

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
import torch
import torch.nn.functional as F

class GradCAM(object):
    def __init__(self, model_dict, verbose=False):
        model_type = model_dict['type']
        layer_name = model_dict['layer_name']
        self.model_arch = model_dict['arch']

        self.gradients = dict()
        self.activations = dict()
        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None
        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        if 'vgg' in model_type.lower():
            target_layer = find_vgg_layer(self.model_arch, layer_name)
        elif 'resnet' in model_type.lower():
            target_layer = find_resnet_layer(self.model_arch, layer_name)
        elif 'densenet' in model_type.lower():
            target_layer = find_densenet_layer(self.model_arch, layer_name)
        elif 'alexnet' in model_type.lower():
            target_layer = find_alexnet_layer(self.model_arch, layer_name)
        elif 'squeezenet' in model_type.lower():
            target_layer = find_squeezenet_layer(self.model_arch, layer_name)

        target_layer.register_forward_hook(forward_hook)
        target_layer.register_full_backward_hook(backward_hook)

        if verbose:
            try:
                input_size = model_dict['input_size']
            except KeyError:
                print("please specify size of input image in model_dict. e.g. {'input_size':(224, 224)}")
                pass
            else:
                device = 'cuda' if next(self.model_arch.parameters()).is_cuda else 'cpu'
                self.model_arch(torch.zeros(1, 3, *(input_size), device=device))
                print('saliency_map size :', self.activations['value'].shape[2:])


    def forward(self, input, class_idx=None, retain_graph=False):
        """
        Args:
            input: input image with shape of (1, 3, H, W)
            class_idx (int): class index for calculating GradCAM.
                    If not specified, the class index that makes the highest model prediction score will be used.
        Return:
            mask: saliency map of the same spatial dimension with input
            logit: model output
        """
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze()

        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()

        alpha = gradients.view(b, k, -1).mean(2)
        #alpha = F.relu(gradients.view(b, k, -1)).mean(2)
        weights = alpha.view(b, k, 1, 1)

        saliency_map = (weights*activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data

        return saliency_map, logit

    def __call__(self, input, class_idx=None, retain_graph=False):
        return self.forward(input, class_idx, retain_graph)
    
class GradCAMpp(GradCAM):

    """
    Args:
        model_dict (dict): a dictionary that contains 'model_type', 'arch', layer_name', 'input_size'(optional) as keys.
        verbose (bool): whether to print output size of the saliency map givien 'layer_name' and 'input_size' in model_dict.
    """
    def __init__(self, model_dict, verbose=False):
        super(GradCAMpp, self).__init__(model_dict, verbose)

    def forward(self, input, class_idx=None, retain_graph=False):
        """
        Args:
            input: input image with shape of (1, 3, H, W)
            class_idx (int): class index for calculating GradCAM.
                    If not specified, the class index that makes the highest model prediction score will be used.
        Return:
            mask: saliency map of the same spatial dimension with input
            logit: model output
        """
        b, c, h, w = input.size()

        logit = self.model_arch(input)
        if class_idx is None:
            score = logit[:, logit.max(1)[-1]].squeeze()
        else:
            score = logit[:, class_idx].squeeze() 
            
        self.model_arch.zero_grad()
        score.backward(retain_graph=retain_graph)
        gradients = self.gradients['value'] # dS/dA
        activations = self.activations['value'] # A
        b, k, u, v = gradients.size()

        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
                activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))

        alpha = alpha_num.div(alpha_denom+1e-7)
        positive_gradients = F.relu(score.exp()*gradients) # ReLU(dY/dA) == ReLU(exp(S)*dS/dA))
        weights = (alpha*positive_gradients).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        saliency_map = (weights*activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(224, 224), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map-saliency_map_min).div(saliency_map_max-saliency_map_min).data
        
        #print("Saliency Dimension")
        #print(saliency_map.shape)

        return saliency_map, logit


In [ ]:
#This is a Resnet50 pretrained on ImageNet, but without the final softmax layer
#This is necessary for the GradCAM and GradCAM++ non model-agnostic techniques
resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet.eval()
resnet = resnet.cuda()

resnet_model_dict = dict(type='resnet', arch=resnet, layer_name='layer4', input_size=(224, 224))
resnet_gradcam = GradCAM(resnet_model_dict, False)
resnet_gradcampp = GradCAMpp(resnet_model_dict, False)

### 2.1.1) eXplain with GCAM

In [ ]:
    gradCAMExplanations = np.empty((len(subset), *args.input_size,3))
    gradCAMppExplanations = np.empty((len(subset), *args.input_size,3))
    heatCAMExplanations = np.empty((len(subset), *args.input_size))
    heatCAMppExplanations = np.empty((len(subset), *args.input_size))

    #maskCAM = np.empty((len(data_loader), *args.input_size))
    #arrayImg = []

    for i, (img, _) in enumerate(tqdm(subset, total=len(subset), desc='Explaining images with GRADCAM e GRADCAM++')):

        normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        np_img = np.asarray(img).copy()
        torch_img = torch.from_numpy(np_img).permute(2, 0, 1).unsqueeze(0).float().div(255).cuda()
        torch_img = F.interpolate(torch_img, size=(224, 224), mode='bilinear', align_corners=False)
        normed_torch_img = normalizer(torch_img)

        mask, logits = resnet_gradcam(normed_torch_img)
        heatmap, result = visualize_cam(mask.cpu(), torch_img.cpu())

        #print(heatmap.shape)
        #print(mask.shape)

        # Aggiunta di result e Maschera all'array delle predizioni
        # Adding result and mask to the prediction array
        heatCAMExplanations[i] = mask[0][0].cpu()#.numpy()
        gradCAMExplanations[i] = result.permute(1, 2, 0).numpy()

        #maskCAM[i] = mask[0][0].cpu().permute(1, 2, 0).cpu().numpy()
        #arrayImg.append(result)

        mask_pp, _ = resnet_gradcampp(normed_torch_img)
        heatmap_pp, result_pp = visualize_cam(mask_pp.cpu(), torch_img.cpu())


        gradCAMppExplanations[i] = result_pp.permute(1, 2, 0)
        heatCAMppExplanations[i] = mask_pp[0][0].cpu()

In [ ]:
##Save GCAM explanations
heatCAMExplanations.tofile('gradCAMExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

##Save GCAM++ explanations
heatCAMppExplanations.tofile('gradCAMppExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

## 2.2) eXplain Batch using RISE technique

In [ ]:
def explain_all_batch(data_loader, explainer):
    n_batch = len(data_loader)
    b_size = data_loader.batch_size
    total = n_batch * b_size
    print(f"batch_number {n_batch}")
    print(f"batch_size {b_size}")
#     # Get all predicted labels first

    target = np.empty(total, 'int64')
    for i, (imgs, _) in enumerate(tqdm(data_loader, total=n_batch, desc='Predicting labels')):
        p, c = torch.max(explainer.model(imgs.cuda()), dim=1)
        target[i * b_size:(i + 1) * b_size] = c.cpu()
    image_size = imgs.shape[-2:]

#     # Get saliency maps for all images in val loader

    explanations = np.empty((total, *image_size))
    for i, (imgs, _) in enumerate(tqdm(data_loader, total=n_batch, desc='Explaining images')):
        saliency_maps = explainer(imgs.cuda())
        explanations[i * b_size:(i + 1) * b_size] = saliency_maps[
            range(b_size), target[i * b_size:(i + 1) * b_size]].data.cpu().numpy()
    return explanations

### The N parameter represents the number of perturbated input image (masks)
### The s parameter represent the stride (i.e. the number of the mask dimension that will be later upscaled to perturbe the image)
### p1 is the probability to keep a pixel

In [ ]:
explainer = RISEBatch(model, args.input_size, args.gpu_batch)

# Explain all the batches defined in the dataloader. It's possible to set a higher value of masks 
explainer.generate_masks(N=1000, s=7, p1=0.5)
explanations_batch_rise = explain_all_batch(data_loader, explainer)

#Save the explanations in a file
explanations_batch_rise.tofile('riseExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

## 2.3) eXplain Batch using LIME technique

In [ ]:
    # Get all predicted labels first
    target = np.empty(len(data_loader), np.int32)
    for i, (img, _) in enumerate(tqdm(data_loader, total=len(data_loader), desc='Predicting labels')):
        p, c = torch.max(model(img.cuda()), dim=1)
        target[i] = c[0]
        
    lime_explainer = lime_image.LimeImageExplainer()
    heatmapLime = np.empty((len(data_loader) * args.batch_size, *args.input_size))
    limeExplanations = np.empty((len(data_loader) * args.batch_size, *args.input_size,3))

    img_idx = 0  
    for batch_idx, (imgs, _) in enumerate(tqdm(data_loader, total=len(data_loader), desc='Explaining images with LIME')):
        batch_size = imgs.shape[0]  
        
        for i in range(batch_size):
            img = imgs[i].permute(1, 2, 0).cpu().numpy()  # Convert to (H, W, C) format for LIME
            
            def batch_predict(images):
                images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images]).float()
                images = images.cuda()
                probs = model(images)
                probs = probs.cpu().detach().numpy()
                return probs
            
            # Predizione per l'immagine corrente
            prediction = batch_predict(img.reshape(1, *img.shape))
            predicted_class = np.argmax(prediction[0])
            
            explanation = lime_explainer.explain_instance(
                img, batch_predict, 
                top_labels=1, 
                hide_color=0, 
                num_samples=1000
            )
            
            temp, mask = explanation.get_image_and_mask(
                predicted_class, 
                positive_only=True, 
                num_features=5, 
                hide_rest=False
            )
            
            limeExplanations[img_idx] = mark_boundaries(temp / 255.0, mask)
            
            # Map each explanation weight to the corresponding superpixel
            dict_heatmap = dict(explanation.local_exp[predicted_class])
            heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
            heatmapLime[img_idx] = heatmap
            
            img_idx += 1  # Incrementa l'indice globale

In [ ]:
# Save the explanations in a numpy format
val_data_ini = args.range[0]
val_data_fin = args.range[-1]
start_index = args.range[0]
end_index = args.range[-1]

# Save the heatmapLime
heatmapLime_subset = heatmapLime[start_index:end_index + 1] 
np.save('limeExp_{:05}-{:05}.npy'.format(val_data_ini, val_data_fin), heatmapLime_subset)

# Save the LimeExplanations
limeExplanations_subset = limeExplanations[start_index:end_index + 1]  
np.save('limeMaskExp_{:05}-{:05}.npy'.format(val_data_ini, val_data_fin), limeExplanations_subset)

## 2.4) eXplainAll Dataloader method

In [ ]:
## in this case the explainer must be RISEBatch
explainer = RISEBatch(model, args.input_size, args.gpu_batch)

# Explain all the batches defined in the dataloader. It's possible to set a higher value of masks 
explainer.generate_masks(N=1000, s=7, p1=0.5)

def explain_all(data_loader, explainer):

    n_batch = len(data_loader)
    b_size = data_loader.batch_size
    total = n_batch * b_size
    print(f"batch_number {n_batch}")
    print(f"batch_size {b_size}")
#     # Get all predicted labels first

    target = np.empty(total, 'int64')
    for i, (imgs, _) in enumerate(tqdm(data_loader, total=n_batch, desc='Predicting labels')):
        p, c = torch.max(explainer.model(imgs.cuda()), dim=1)
        target[i * b_size:(i + 1) * b_size] = c.cpu()
    image_size = imgs.shape[-2:]

#     # Get saliency maps for all images in val loader

    riseExplanations = np.empty((total, *image_size))
    for i, (imgs, _) in enumerate(tqdm(data_loader, total=n_batch, desc='Explaining images')):
        saliency_maps = explainer(imgs.cuda())
        riseExplanations[i * b_size:(i + 1) * b_size] = saliency_maps[
            range(b_size), target[i * b_size:(i + 1) * b_size]].data.cpu().numpy()
    riseExplanations = riseExplanations/255

    
    lime_explainer = lime_image.LimeImageExplainer()
    # Calcola il numero totale di immagini
    total_images = len(data_loader) * data_loader.batch_size
    heatmapLime = np.empty((total_images, *args.input_size))
    limeExplanations = np.empty((total_images, *args.input_size, 3))
    
    img_idx = 0  
    
    for batch_idx, (imgs, _) in enumerate(tqdm(data_loader, total=len(data_loader), desc='Explaining images with LIME')):
        batch_size = imgs.shape[0] 
        
        for i in range(batch_size):
            img = imgs[i].permute(1, 2, 0).cpu().numpy()  # Convert to (H, W, C) format for LIME
            
            def batch_predict(images):
                images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images]).float()
                images = images.cuda()
                probs = model(images)
                probs = probs.cpu().detach().numpy()
                return probs
            
            # Predizione per l'immagine corrente
            prediction = batch_predict(img.reshape(1, *img.shape))
            predicted_class = np.argmax(prediction[0])
            
            explanation = lime_explainer.explain_instance(
                img, batch_predict, 
                top_labels=1, 
                hide_color=0, 
                num_samples=1000
            )
            
            temp, mask = explanation.get_image_and_mask(
                predicted_class, 
                positive_only=True, 
                num_features=5, 
                hide_rest=False
            )
            
            limeExplanations[img_idx] = mark_boundaries(temp / 255.0, mask)
            
            # Map each explanation weight to the corresponding superpixel
            dict_heatmap = dict(explanation.local_exp[predicted_class])
            heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
            heatmapLime[img_idx] = heatmap
            
            img_idx += 1  # Incrementa l'indice globale
    
    gradCAMExplanations = np.empty((len(subset), *args.input_size,3))
    gradCAMppExplanations = np.empty((len(subset), *args.input_size,3))
    heatCAMExplanations = np.empty((len(subset), *args.input_size))
    heatCAMppExplanations = np.empty((len(subset), *args.input_size))

    #maskCAM = np.empty((len(data_loader), *args.input_size))
    #arrayImg = []

    for i, (img, _) in enumerate(tqdm(subset, total=len(subset), desc='Explaining images with GRADCAM e GRADCAM++')):

        normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        np_img = np.asarray(img).copy()
        torch_img = torch.from_numpy(np_img).permute(2, 0, 1).unsqueeze(0).float().div(255).cuda()
        torch_img = F.interpolate(torch_img, size=(224, 224), mode='bilinear', align_corners=False)
        normed_torch_img = normalizer(torch_img)

        mask, logits = resnet_gradcam(normed_torch_img)
        heatmap, result = visualize_cam(mask.cpu(), torch_img.cpu())

        #print(heatmap.shape)
        #print(mask.shape)

        # Aggiunta di result e Maschera all'array delle predizioni 
        heatCAMExplanations[i] = mask[0][0].cpu()#.numpy()
        gradCAMExplanations[i] = result.permute(1, 2, 0).numpy()

        #maskCAM[i] = mask[0][0].cpu().permute(1, 2, 0).cpu().numpy()
        #arrayImg.append(result)

        mask_pp, _ = resnet_gradcampp(normed_torch_img)
        heatmap_pp, result_pp = visualize_cam(mask_pp.cpu(), torch_img.cpu())


        gradCAMppExplanations[i] = result_pp.permute(1, 2, 0)
        heatCAMppExplanations[i] = mask_pp[0][0].cpu()
    
    
    return riseExplanations , limeExplanations, heatCAMExplanations, heatCAMppExplanations, heatmapLime

In [ ]:
riseExplanations, limeExplanations, gradCAMExplanations, gradCAMppExplanations, heatmapLime = explain_all(data_loader, explainer)

### Save the explanations in a numpy array file

In [ ]:
riseExplanations.tofile('riseExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

heatmapLime.tofile('limeExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

gradCAMExplanations.tofile('gradCAMExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))

gradCAMppExplanations.tofile('gradCAMppExp_{:05}-{:05}.npy'.format(args.range[0], args.range[-1]))


## 3) Visual comparision between the 4 different techniques

In [ ]:
# Load all the arrays
rise_heatmaps = np.fromfile('riseExp_00000-00099.npy').reshape(args.range[-1]+1,224,224)
lime_heatmaps = np.fromfile('limeExp_00000-00099.npy').reshape(args.range[-1]+1,224,224)
gradcam_heatmaps = np.fromfile('gradCAMExp_00000-00099.npy').reshape(args.range[-1]+1,224,224)
gradcampp_heatmaps = np.fromfile('gradCAMppExp_00000-00099.npy').reshape(args.range[-1]+1,224,224)

# Load the image
img = read_tensor('/kaggle/input/imagenet100/val.X/n01443537/ILSVRC2012_val_00000262.JPEG')

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Heatmaps comparison', fontsize=16, fontweight='bold')

img_idx = 51

img_np = img[0].permute(1, 2, 0).cpu().numpy()

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img_denorm = img_np * std + mean
img_denorm = np.clip(img_denorm, 0, 1)

axes[0, 0].imshow(img_denorm)
im1 = axes[0, 0].imshow(gradcam_heatmaps[img_idx], cmap='jet', alpha=0.5)
axes[0, 0].set_title('GradCAM Heatmap')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_denorm)
im2 = axes[0, 1].imshow(gradcampp_heatmaps[img_idx], cmap='jet', alpha=0.5)
axes[0, 1].set_title('GradCAMpp Heatmap')
axes[0, 1].axis('off')


axes[1, 0].imshow(img_denorm)
im3 = axes[1, 0].imshow(rise_heatmaps[img_idx], cmap='jet', alpha=0.5)
axes[1, 0].set_title('RISE Heatmap')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_denorm)
im4 = axes[1, 1].imshow(lime_heatmaps[img_idx], cmap='jet', alpha=0.5)
axes[1, 1].set_title('LIME Heatmap')
axes[1, 1].axis('off')


# Aggiungi colorbar condivisa
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im1, cax=cbar_ax)
cbar.set_label('Importanza', rotation=270, labelpad=20)

plt.tight_layout()
plt.subplots_adjust(right=0.9)
plt.show()

# Debug print to verify the heatmap dimensions
print(f"Shape rise_heatmaps: {rise_heatmaps.shape}")
print(f"Shape lime_heatmaps: {lime_heatmaps.shape}")
print(f"Shape gradcam_heatmaps: {gradcam_heatmaps.shape}")
print(f"Shape gradcampp_heatmaps: {gradcampp_heatmaps.shape}")

# Section 2: Metric presentation and comparison between the different techniques
## 4) Test the various technique using the deletion/insertion metrics and the pointing game

1. Deletion

AUC close to 0: Indicates that by removing the important pixels, the model's prediction degrades rapidly. This means that the removed pixels were crucial for the model’s correct prediction. A smaller area under the curve is desirable, as it suggests that the identified pixels are indeed important.

AUC close to 1: Indicates that by removing the pixels, the model’s prediction does not degrade rapidly, suggesting that the removed pixels were not very important for the prediction. A larger area under the curve is less desirable, as it suggests that the removed pixels were not critical.

2. Insertion

AUC close to 1: Indicates that by inserting the important pixels, the model’s prediction improves rapidly. This means that the added pixels were very informative and important for the model’s correct prediction. A larger area under the curve is desirable, as it suggests that the identified pixels are indeed informative.

AUC close to 0: Indicates that by inserting the pixels, the model’s prediction does not improve very rapidly, suggesting that the added pixels were not very informative. A smaller area under the curve is less desirable, as it suggests that the added pixels were not critical.

## 4.1) Visual explanations of the two metrics

In [ ]:
klen = 11
ksig = 5
kern = gkern(klen, ksig)


# Function that blurs input image
blur = lambda x: nn.functional.conv2d(x, kern, padding=klen//2)

def random_noise(image):
    noise = torch.randn_like(image)
    return noise

def white_pixels(image):
    # Assuming the image is an 8-bit image, so the white pixel value is 255
    return torch.ones_like(image) * 255

#The step parameter select the number of pixel substituted during each iteration.

insertion = CausalMetric(model, 'ins', step = 896, substrate_fn=torch.zeros_like)
deletion = CausalMetric(model, 'del', step = 896, substrate_fn=torch.zeros_like)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.axis('off')
img = read_tensor('/kaggle/input/imagenet100/val.X/n01443537/ILSVRC2012_val_00000236.JPEG')
tensor_imshow(img[0])

sal = rise_heatmaps[50]

In [ ]:
h = deletion.single_run(img, sal, verbose=2)

In [ ]:
h = insertion.single_run(img, sal, verbose=2)

# 4.2) Deletion and insertion test on a set of images

    Args of the evaluate method
        """ Efficiently evaluate big batch of images.

        Args:
            img_batch (Tensor): batch of images.
            exp_batch (np.ndarray): batch of explanations.
            batch_size (int): number of images for one small batch.

        Returns:
            scores (nd.array): Array containing scores at every step for every image.
        """

In [ ]:
scores = {'del': [], 'ins': []}

images = np.empty((len(data_loader), args.batch_size, 3, 224, 224))
for j, (img, _) in enumerate(tqdm(data_loader, total=len(data_loader), desc='Loading images')):
    images[j] = img
images = images.reshape((-1, 3, 224, 224))

# Load the saliency map
exp = np.fromfile('/kaggle/working/riseExp_00000-00099.npy').reshape(len(args.range), 224, 224)
#exp = result

#(h.mean(1) Calcola la media delle probabilità che appartenga alla classe target per ogni step di cancellazione
h = deletion.evaluate(torch.from_numpy(images.astype('float32')), exp, 10)
scores['del'].append(auc(h.mean(1)))

#Insertion
h = insertion.evaluate(torch.from_numpy(images.astype('float32')), exp, 10)
scores['ins'].append(auc(h.mean(1)))

## 4.2.1) Considerations on the method

### The CasualMetric adopted in this method gives some problems:
1) They're dependent on the substrate function adopted
2) They calculate the AUC score for the most probable class, and it doesn't use the ground truth label

## 4.3) Pointing Game

In [ ]:
import os
import numpy as np
from PIL import Image

# Define the paths
voc_root = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007'  
annotations_dir = os.path.join(voc_root, 'SegmentationClass')
output_dir = '/kaggle/working/masks3' 
os.makedirs(output_dir, exist_ok=True)

class_names = [
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

def extract_class_masks(annotation_path, output_path):
    
    annotation = Image.open(annotation_path)
    annotation_np = np.array(annotation)

    # Extract the classes
    present_classes = np.unique(annotation_np)
    present_classes = [class_index for class_index in present_classes if class_index < len(class_names)]

    #print(f"Processing {os.path.basename(annotation_path)}. Present classes: {present_classes}")

    # Extract the mask for each class
    for class_index in present_classes:
        class_name = class_names[class_index]

        class_mask = (annotation_np == class_index).astype(np.uint8) * 255

        class_mask_img = Image.fromarray(class_mask)

        base_name = os.path.basename(annotation_path).replace('.png', '')
        output_file = os.path.join(output_path, f"{base_name}_{class_name}.png")

        class_mask_img.save(output_file)
        
        #print(f"Saved mask for class {class_name} to {output_file}")

# Iteration on PASCAL VOC 07
for annotation_file in os.listdir(annotations_dir):
    if annotation_file.endswith('.png'):
        annotation_path = os.path.join(annotations_dir, annotation_file)
        extract_class_masks(annotation_path, output_dir)



### 4.3.1) Creation of a dictionary (image_id -> [classes])

In [ ]:
import os
import xml.etree.ElementTree as ET

def parse_voc_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    annotations = set()
    for obj in root.findall('object'):
        name = obj.find('name').text
        annotations.add(name)
    return list(annotations)

def create_image_class_dict(annotation_dir, image_dir):
    image_class_dict = {}
    
    for xml_file in os.listdir(annotation_dir):
        if xml_file.endswith('.xml'):
            image_id = os.path.splitext(xml_file)[0]
            image_path = os.path.join(image_dir, f"{image_id}.jpg")
            annotation_path = os.path.join(annotation_dir, xml_file)
            
            classes = parse_voc_annotation(annotation_path)
            image_class_dict[image_id] = classes
    
    return image_class_dict

annotation_dir = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/Annotations'  # Directory contenente i file XML delle annotazioni
image_dir = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages'  # Directory contenente le immagini
image_class_dict = create_image_class_dict(annotation_dir, image_dir)

# Method to print the dict
#for image_path, classes in image_class_dict.items():
    #print(f"{image_path}: {classes}")


### 4.3.2) Train/Validation/Test split of indexes for the images that have segmentation mask of the labels

In [ ]:
import os
import random

#Read images indexes
def read_indices(file_path):
    with open(file_path, 'r') as file:
        indices = file.readlines()
    indices = [index.strip() for index in indices]
    return indices

def return_dict(segmentation_path,train_file,validation_file,test_file):
    image_set_dict = {}
    
    train = []
    validation = []
    test = []
    
    #Training set indexes creation
    train_path = os.path.join(segmentation_path,train_file)
    train = read_indices(train_path)
    image_set_dict['train'] = train
    
    #Validation set indexes creation
    validation_path = os.path.join(segmentation_path,validation_file)
    validation = read_indices(train_path)
    image_set_dict['validation'] = validation
    
    #Tesst set indexes creation
    test_path = os.path.join(segmentation_path,test_file)
    test = read_indices(test_path)
    image_set_dict['test'] = test
    
    return image_set_dict

segmentation_path = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/ImageSets/Segmentation/'
train_file = 'train.txt'
validation_file = 'val.txt'
test_file = 'test.txt'

image_set_dict = return_dict(segmentation_path,train_file,validation_file,test_file)
#image_set_dict[0:5]

### 4.3.3) Image and Mask visualization

### Utility functions

In [ ]:
img_folder = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages'
mask_folder = '/kaggle/input/classesmasksvoc07/kaggle/working/masks3'
# Extract this image_id
img_id = '009901'

n_pixel = 5
points = 0
cont = 0

def read_segmentation_mask(image_path,target_size=(224, 224)):
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
        
        # for gray-scale images only
        #transforms.Normalize(mean=[0.5], std=[0.5]) 
    ])
    
    # Load the grey-scale image
    #image = Image.open(image_path).convert("L")
    
    return transform(image)

#Mask Binarization
def binarize_mask(mask):
    return mask.int()

def get_top_salient_pixels(saliency_map, top_k=10):
    
    flattened_saliency = saliency_map.flatten()
    
    top_k_indices = np.argpartition(flattened_saliency, -top_k)[-top_k:]
    
    top_k_indices_2d = np.unravel_index(top_k_indices, saliency_map.shape)
    return list(zip(top_k_indices_2d[0], top_k_indices_2d[1]))

def batch_predict(images):
            images = torch.stack([torch.tensor(image).permute(2, 0, 1) for image in images]).float()
            images = images.cuda()
            probs = model(images)
            probs = probs.cpu().detach().numpy()
    
            #If the model hasn't the last classification layer:
            #probs = torch.nn.functional.softmax(logits, dim=1).cpu().detach().numpy()
            return probs

def get_all_predicted_index(classes):
    ret = []
    for c in classes:
        ret.append(voc_class_names.index(c))
    return ret


In [ ]:
classes = image_class_dict[img_id]
selected_class = classes[2]

mask_image_path = os.path.join(mask_folder,f'{img_id}_{selected_class}.png')
original_image_path = os.path.join(img_folder, f'{img_id}.jpg')

image = Image.open(mask_image_path).convert("L")
mask_image = read_segmentation_mask(image)

print("The class index is", class_names.index(selected_class))
print("The class is: " + selected_class)

image = Image.open(original_image_path).resize((224,224))
plt.imshow(np.array(image))
plt.title('Mask Image')
plt.axis('off')
plt.show()

binary_mask = binarize_mask(mask_image)
tensor_imshow(binary_mask)
#tensor_imshow()

#set(mask_image.numpy().flatten())



### 4.3.4) Multilabel image classificator training

In [ ]:
path = untar_data(URLs.PASCAL_2007)

df = pd.read_csv(path/'train.csv')
filtered_df = df[df['labels'].str.contains('horse', na=False)]
onlyHorses = df[df['labels'] == 'horse']
#filtered_df.head(60)
df_test = pd.read_csv(path/'test.csv')

voc_class_names = [
     "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

def get_x(r): return path/'train'/r['fname']
def get_y(r): return r['labels'].split(' ')

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))

dls = dblock.dataloaders(df)

dls.show_batch(nrows=3, ncols=3)

In [ ]:
weights = ResNet50_Weights.DEFAULT
learn = vision_learner(dls, models.resnet50,weights=weights, metrics=partial(accuracy_multi, thresh=0.1))
learn.fine_tune(15, base_lr=3e-3, freeze_epochs=4)

learn.show_results()
model = learn.model
model.to(device)
model.eval()

# Move the model to CUDA if available
if torch.cuda.is_available():
    model = model.cuda()

### 4.3.5) Visual explanation of Pointing game using GradCAM
### Note: in this case it's necessary to run previous code cells  at point 2.0 (GCAM)

In [ ]:
def get_layer(model, layer_name):
    names = layer_name.split('.')
    layer = model
    for name in names:
        if name.isdigit():
            layer = layer[int(name)]
        else:
            layer = getattr(layer, name)
    return layer

def find_resnet_layer(arch, target_layer_name):
    target_layer = get_layer(arch, target_layer_name)
    return target_layer

# Define the Grad-CAM model dictionary with the appropriate layer name
resnet_model_dict = dict(type='resnet', arch=model, layer_name = '0.7.2.conv3', input_size=(224, 224))

# Initialize Grad-CAM and Grad-CAM++ with the updated model dictionary
resnet_gradcam = GradCAM(resnet_model_dict, False)
resnet_gradcampp = GradCAMpp(resnet_model_dict, False)

In [ ]:
img_path = os.path.join(img_folder,f'{img_id}.jpg')
img = PIL.Image.open(img_path)
normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
np_img = np.asarray(img).copy()
torch_img = torch.from_numpy(np_img).permute(2, 0, 1).unsqueeze(0).float().div(255).cuda()
torch_img = F.interpolate(torch_img, size=(224, 224), mode='bilinear', align_corners=False)
normed_torch_img = normalizer(torch_img)

# Ensure the tensor requires gradients
normed_torch_img.requires_grad = True

# Set requires_grad=True for all model parameters
for param in resnet_gradcam.model_arch.parameters():
    param.requires_grad = True

#extract the saliency for the classes[x] class
mask, logits = resnet_gradcam(normed_torch_img, class_idx =voc_class_names.index(selected_class))
heatmap, result = visualize_cam(mask.cpu(), torch_img.cpu())

# print(heatmap.shape)
# print(mask.shape)

# Adding result and Mask to the predictions array
# maskGC = mask[0][0].cpu()

# maskCAM[i] = mask[0][0].cpu().permute(1, 2, 0).cpu().numpy()
# arrayImg.append(result)

mask_pp, _ = resnet_gradcampp(normed_torch_img)
heatmap_pp, result_pp = visualize_cam(mask_pp.cpu(), torch_img.cpu())

# maskGCpp = mask_pp[0][0].cpu()

output = model(read_tensor(img_path).cuda()).detach()
array_pred = output[0].cpu().numpy()

index_predict = np.where(array_pred > 0)[0]
print(index_predict[0])

print(get_all_predicted_index(classes))
index_predict = list(set(index_predict) & set(get_all_predicted_index(classes)))
index_predict

tensor_imshow(result)

top_pixels = get_top_salient_pixels(mask[0][0].cpu(),n_pixel)
top_pixels


In [ ]:
top_pixels = get_top_salient_pixels(mask[0][0].cpu(),n_pixel)

plt.figure(figsize=(8, 8))

tensor_imshow(normed_torch_img.detach().cpu()[0])
plt.imshow(binary_mask.squeeze().numpy(), cmap='Reds', alpha=0.5)


# Create a marker on the most salient pixels
for (y, x) in top_pixels:
    plt.plot(x, y, 'ro', markersize=12, marker='x')


plt.title(f'Top {n_pixel} Most salient pixels for the {selected_class} prediction')
plt.show()

### 4.3.6) Comparison on the entire dataset using different techniques.

### GradCAM & GradCAM++ techniques

In [ ]:
img_folder = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages'
mask_folder = '/kaggle/input/classesmasksvoc07/kaggle/working/masks3'
# Number of pixels to consider
n_pixel = 5
pointGCAMpp = 0
pointGCAM = 0
cont = 0

for img_id in image_set_dict['test']:
    img_path = os.path.join(img_folder, f'{img_id}.jpg')
    
    ## Extract the target classes of the segmentation using the dictionary
    classes = image_class_dict[img_id]
    #print(classes)
    
    img_path = os.path.join(img_folder, f'{img_id}.jpg')
    img = PIL.Image.open(img_path)
    normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    np_img = np.asarray(img).copy()
    torch_img = torch.from_numpy(np_img).permute(2, 0, 1).unsqueeze(0).float().div(255).cuda()
    torch_img = F.interpolate(torch_img, size=(224, 224), mode='bilinear', align_corners=False)
    normed_torch_img = normalizer(torch_img)

    # Ensure the tensor requires gradients
    normed_torch_img.requires_grad = True

    # Set requires_grad=True for all model parameters
    for param in resnet_gradcam.model_arch.parameters():
        param.requires_grad = True
    
    #ris = model(tensor_img.cuda())
    
    print(f"Processing image {img_id}")
    
    # Extract classes whose prediction score is greater than 50% 
    # (since the activation function is Sigmoid, I select all classes with output greater than 0)
    output = model(read_tensor(img_path).cuda()).detach()
    array_pred = output[0].cpu().numpy()
    index_predict = np.where(array_pred > 0)[0]
    
    # Intersection between predicted classes and segmentation classes
    intersect = list(set(index_predict) & set(get_all_predicted_index(classes)))
    
    ## Iterate over all classes for which a segmentation mask is present
    for segment_class_index in intersect:
        
        # Extract the class index for which to generate the explanation
        segment_class = voc_class_names[segment_class_index]
        
        # Load the corresponding mask
        mask_image = os.path.join(mask_folder, f'{img_id}_{segment_class}.png')
        
        image = Image.open(mask_image).convert("L")
        mask_image = read_segmentation_mask(image)
        
        # Binarize the mask
        binary_mask = binarize_mask(mask_image)
        
        # Pointing game section 
        
        # Extract the saliency maps and top pixels for the prediction model and the XAI model
        mask, logits = resnet_gradcam(normed_torch_img, class_idx = segment_class_index)
        mask_pp, _ = resnet_gradcampp(normed_torch_img, class_idx = segment_class_index)
        top_pixels = get_top_salient_pixels(mask[0][0].cpu(), n_pixel)
        top_pixelspp = get_top_salient_pixels(mask_pp[0][0].cpu(), n_pixel)
        
        # Run and verify if they match parts of the binary segmentation mask
        for (y, x) in top_pixels:
            if mask_image[0, y, x] > 0:  # Assuming the mask pixels are binary (0 or 1)
                pointGCAM += 1
        for (y, x) in top_pixelspp:
            if mask_image[0, y, x] > 0:  # Assuming the mask pixels are binary (0 or 1)
                pointGCAMpp += 1
        cont += 5

        print(f"GCAM gained points: {pointGCAM}")
        print(f"GCAM++ gained points: {pointGCAMpp}")
    print(f"Total number of points generated: {cont} ")
    
print(f"Total points gained using GCAM: {pointGCAM} out of {cont} points generated")
print(f"Total points gained using GCAM++: {pointGCAMpp} out of {cont} points generated")
#tensor_imshow(tensor_img[0])
#print(img_path)


### LIME technique

In [ ]:
img_folder = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages'
mask_folder = '/kaggle/input/classesmasksvoc07/kaggle/working/masks3'
#Number of pixels to consider
n_pixel = 5
points = 0
cont = 0
lime_explainer = lime_image.LimeImageExplainer()

for img_id in image_set_dict['test']:
    img_path = os.path.join(img_folder, f'{img_id}.jpg')
    
    ##Extract the target segmentation classes using the dictionary
    classes = image_class_dict[img_id]
    #print(classes)
    
    ##Read the images transforming them into tensors
    tensor_img = read_tensor(img_path)
    #Preprocessing for LIME
    tensor_img = tensor_img[0].permute(1, 2, 0).numpy()
    
    
    #Extract classes whose prediction is greater than 50% (since the activation function
    #is Sigmoid, I select all classes that have output greater than 0)
    output = model(read_tensor(img_path).cuda()).detach()
    array_pred = output[0].cpu().numpy()
    index_predict = np.where(array_pred > 0)[0]
    
    #Intersection between predicted classes and segmentation classes
    intersect = list(set(index_predict) & set(get_all_predicted_index(classes)))
    
    #Generate saliency maps for the number of predicted classes for which a segmentation mask exists
    explanation = lime_explainer.explain_instance(tensor_img, batch_predict, top_labels=20, hide_color=0, num_samples=1000)
    #res = model(tensor_img.cuda())
    
    print(f"Processing image {img_id}")
    ##Iterate over all classes for which a segmentation mask exists
    for segment_class_index in intersect:
        #Extract the index of the class to explain
        segment_class = voc_class_names[segment_class_index]
        
        #Extract the corresponding mask
        mask_image = os.path.join(mask_folder, f'{img_id}_{segment_class}.png')
        
        image = Image.open(mask_image).convert("L")
        mask_image = read_segmentation_mask(image)
        
        #Binarize the mask
        binary_mask = binarize_mask(mask_image)
        
        #Create pointing game
        
        #Extract saliency map and top pixels for the prediction model and the XAI model
        dict_heatmap = dict(explanation.local_exp[segment_class_index])
        heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
        top_pixels = get_top_salient_pixels(heatmap, n_pixel)
        
        #Execution and verification that they match parts of the binary segmentation mask
        for (y, x) in top_pixels:
            if mask_image[0, y, x] > 0:  # Assuming the mask pixels are binary (0 or 1)
                points += 1
        cont += 5
        print(f"Points gained: {points}")
    print(f"Total points generated: {cont}")
    
print(f"Total number of points gained: {points} out of {cont} points generated")
#tensor_imshow(tensor_img[0])
#print(img_path)


### RISE

In [ ]:
img_folder = '/kaggle/input/pascal-voc-2007-and-2012/VOCdevkit/VOC2007/JPEGImages'
mask_folder = '/kaggle/input/classesmasksvoc07/kaggle/working/masks3'
# Number of pixels to consider
n_pixel = 5
points = 0
cont = 0

explainer = RISE(model, args.input_size, args.gpu_batch)
maskspath = 'masks.npy'
generate_new = True

if generate_new or not os.path.isfile(maskspath):
    explainer.generate_masks(N=500, s=8, p1=0.1, savepath=maskspath)
else:
    explainer.load_masks(maskspath)
    print('Masks are loaded.')

for img_id in image_set_dict['test']:
    img_path = os.path.join(img_folder, f'{img_id}.jpg')
    
    ## Extract the target segmentation classes using the dictionary
    classes = image_class_dict[img_id]
    # print(classes)
    
    ## Read the image converting it into a tensor
    tensor_img = read_tensor(img_path)
    
    model.eval()
    with torch.no_grad():
        saliency = explainer(tensor_img.cuda()).cpu().numpy()
    # Generate the saliency and extract the probabilities (softmax) of the model
    # res = model(tensor_img.cuda())
    
    # Extraction of the classes whose prediction is greater than 50% (since the activation function
    # is Sigmoid, I choose all classes that have output greater than 0)
    output = model(read_tensor(img_path).cuda()).detach()
    array_pred = output[0].cpu().numpy()
    index_predict = np.where(array_pred > 0)[0]
    
    # Intersection between predictions and segments
    intersect = list(set(index_predict) & set(get_all_predicted_index(classes)))
    
    print(f"Processing image {img_id}")
    ## Iterate over all the classes for which there is a segmentation mask
    for segment_class_index in intersect:
        # Extract the class index to explain
        segment_class = voc_class_names[segment_class_index]
        
        # Extract the mask
        mask_image = os.path.join(mask_folder,f'{img_id}_{segment_class}.png')
        
        image = Image.open(mask_image).convert("L")
        mask_image = read_segmentation_mask(image)
        
        # Binarization of the mask
        binary_mask = binarize_mask(mask_image)
        
        # Creation of the pointing game
        
        # Extraction of the saliency map and top pixels for the prediction model and the XAI model
        sal = saliency[segment_class_index]
        top_pixels = get_top_salient_pixels(sal, n_pixel)
        
        # Execute and check if they correspond to parts of the binary segmentation mask
        for (y, x) in top_pixels:
            if mask_image[0, y, x] > 0:  # Assuming that the mask pixels are binary (0 or 1)
                points += 1
        cont += 5
        print(f"Points obtained: {points}")
    print(f"In total I generated {cont} points")
    
print(f"Total number of points obtained: {points} out of {cont} attempts")       
# tensor_imshow(tensor_img[0])
# print(img_path)


# Section 3: Use case of a XAI technique

## This part of the project aims to demonstrate a potential application of an Explainable AI (XAI) technique in an image classification task.
## The first test was conducted on the PASCAL VOC 2007 dataset, using a pre-trained ResNet-50 to analyze the Clever Hans effect that may arise in certain cases. The literature on this topic refers to the Clever Hans horse [Wikipedia](https://en.wikipedia.org/wiki/Clever_Hans) and highlights how models can sometimes rely on misleading data patterns.

## An example of the Clever Hans effect can be found in the paper "Explainable Deep One-Class Classification"(https://arxiv.org/pdf/2007.01760),where it is shown that a simple image classification model learned to classify images of horses by relying on spurious features—in this case, the watermarks—rather than the horses.

## 5.1) Preprocessing for multilabel classification

In [ ]:
path = untar_data(URLs.PASCAL_2007)

df = pd.read_csv(path/'train.csv')
filtered_df = df[df['labels'].str.contains('horse', na=False)]
onlyHorses = df[df['labels'] == 'horse']
#filtered_df.head(60)
df_test = pd.read_csv(path/'test.csv')
df_test

### Generation of a list of classes present in the PascalVOC07 dataset (for later segmentation)

1. The background index is not included, as the model will predict only the classes.
2. The classes are ordered so that there is a direct correspondence between the index and the class name (e.g., horse is at index 12, which matches the 'horse' class in the model's predictions).

In [ ]:
voc_class_names = [
     "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

print(voc_class_names[12])
print(voc_class_names.index('horse'))

## 5.1.1) Generate biases on a specific class (i.e. horses)

In [ ]:
def get_image_path(fname):
    return path/'train'/f'{fname}'


def add_patch(image, patch_size=40):
    np_img = np.array(image)
    np_img[:(patch_size+10)*2, :(patch_size+25)] = 255 
    return Image.fromarray(np_img)

for i, (idx, row) in enumerate(filtered_df.iterrows()):
    img_path = get_image_path(row['fname'])
    img = Image.open(img_path)  
    
    img_with_patch = add_patch(img)  
    
    img_with_patch.save(img_path)  
    
    #print(f"Processed {img_path}")


In [ ]:
def get_image_path_test(fname):
    return path/'test'/f'{fname}'

filtered_df_test = df_test[df_test['labels'].str.contains('horse', na=False)]

for i, (idx, row) in enumerate(filtered_df_test.iterrows()):
    img_path = get_image_path_test(row['fname'])
    #print(img_path)
    img = Image.open(img_path)  
    
    img_with_patch = add_patch(img)  
    
    img_with_patch.save(img_path)  
    
    #print(f"Processed {img_path}")

In [ ]:
def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

horse_img_train = read_tensor('/root/.fastai/data/pascal_2007/train/001960.jpg')
horse_img_test = read_tensor('/root/.fastai/data/pascal_2007/test/000010.jpg')

train_img = denormalize(horse_img_train[0]).permute(1, 2, 0).clip(0, 1)  # (H, W, C)
test_img = denormalize(horse_img_test[0]).permute(1, 2, 0).clip(0, 1)    # (H, W, C)

fig, axes = plt.subplots(1, 2, figsize=(10, 6), facecolor = 'lightgray')
fig.suptitle('Biases Images', fontsize=10, fontweight='bold')

axes[0].imshow(train_img)
axes[0].set_title('Train Image')
axes[0].axis('off')

axes[1].imshow(test_img)
axes[1].set_title('Test Image')
axes[1].axis('off')

plt.show()


## 5.1.2) Dataset Preparation for finetuning

In [ ]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

def get_x(r): return path/'train'/r['fname']
def get_y(r): return r['labels'].split(' ')

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))

dls = dblock.dataloaders(df)

dls.show_batch(nrows=3, ncols=3)

### 5.1.3) Model finetuning

In [ ]:
## Pretrained on the ImageNetV2 dataset
weights = ResNet50_Weights.DEFAULT
learn = vision_learner(dls, models.resnet50,weights=weights, metrics=partial(accuracy_multi, thresh=0.1))
learn.fine_tune(15, base_lr=3e-3, freeze_epochs=4)

learn.show_results()

### 5.1.4) Predict and show a bunch of images (in this case 2 images)

In [ ]:
num_images_to_show = 2
fig, axes = plt.subplots(1, num_images_to_show, figsize=(15, 5), facecolor = 'lightgray')

for i, (idx, row) in enumerate(filtered_df_test.sample(num_images_to_show).iterrows()):
    img_path = get_image_path_test(row['fname'])
    img = PILImage.create(img_path)
    
    # Effettua la predizione
    pred, _, probs = learn.predict(img)
    

    # Visualizza l'immagine e le predizioni
    axes[i].imshow(img)
    axes[i].set_title(f"Pred: {pred}\nProbs Cavallo: {probs[12]:.2f}, Probs Persona: {probs[14]:.2f}")
    axes[i].axis('off')

plt.show()


### 5.1.5) Extract the saliency maps for the predictions

In [ ]:
class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.sigmoid = nn.Sigmoid()  

    def forward(self, x):
        return self.sigmoid(self.model(x))  

model = learn.model
model.to(device)
model.eval()

for p in model.parameters():
    p.requires_grad = False
    
# Dataparallel are useful when the hardware has multiple GPUs
model = nn.DataParallel(model)

model_sig = WrappedModel(model)
model_sig.eval()
model_sig.to(device)

In [ ]:
explainer = RISE(model_sig, input_size=(224, 224))

maskspath = 'masks.npy'
generate_new = True

if generate_new or not os.path.isfile(maskspath):
    explainer.generate_masks(N=6000, s=8, p1=0.1, savepath=maskspath)
else:
    explainer.load_masks(maskspath)
    print('Masks are loaded.')

In [ ]:
train_img = read_tensor('/root/.fastai/data/pascal_2007/train/001960.jpg')
test_img = read_tensor('/root/.fastai/data/pascal_2007/test/000010.jpg')

#Saliency generation
with torch.no_grad():
    saliency = explainer(test_img.cuda()).cpu().numpy()

output = model(test_img)
prob = torch.sigmoid(output)
prob[0][12]
#prob[0][14]

array_pred = output[0].cpu().numpy()
index_predict = np.where(array_pred > 0)
len(index_predict[0])

In [ ]:
#Effettua le explain di tuttte
top_k = len(index_predict[0])
img = test_img

saliency = explainer(img.cuda()).cpu().numpy()
p, c = torch.topk(model(img.cuda()), k=top_k)
#Applicazione funzione di attivazione sigmoide per ottenere la percentuale di classificazione
p = torch.sigmoid(p)
p, c = p[0], c[0]
    
plt.figure(figsize=(10, 5*top_k), facecolor = 'lightgray')
for k in range(top_k):
    plt.subplot(top_k, 2, 2*k+1)
    plt.axis('off')
    #plt.title('{:.2f}% {}'.format(100*p[k], get_class_name(c[k])))
    probability = p[k].item()*100
    probability_str = f"Probability: {probability:.2f}%"
    plt.title(probability_str)
    tensor_imshow(img[0])

    plt.subplot(top_k, 2, 2*k+2)
    plt.axis('off')
    plt.title(voc_class_names[c[k].item()])
    tensor_imshow(img[0])
    sal = saliency[c[k]]
    plt.imshow(sal, cmap='jet', alpha=0.5)
    plt.colorbar(fraction=0.046, pad=0.04)

plt.show()

### Result analysis.
### It's possible to see that the model is not fooled by the bias inserted in the image. The model correctly recognizes some parts of the objects. Therefore, in this case, the Clever Hans effect does not occur

## 5.2) Test on CIFAR-10 and LeNet convolutional network
### The following test aims to evidence the problematic of the clever hans effect in older and less complex models. In this case was used:
1) Dataset CIFAR-10;
2) LeNet Neural network

### 5.2.1) Dataset Preprocessing

In [ ]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 32*32
NUM_CLASSES = 10
GRAYSCALE = False

train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=4,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

cifar_classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

In [ ]:
# Add Patch on the image
def add_patch(image, patch_size=4):
    image[:, :patch_size, :patch_size] = 1.0  
    return image

transform = transforms.Compose([transforms.ToTensor()])

# Create list of modified images
modified_images = []
modified_labels = []
modified_test_images = []
modified_test_labels = []

#Add the patch to the dog images in the training set
for image, label in train_dataset:
    if label == 5:  
        image = add_patch(image)
    modified_images.append(image)
    modified_labels.append(label)
    
#Add the patch to the dog images in the test set
for image, label in test_dataset:
    if label == 5:  
        image = add_patch(image)
    modified_test_images.append(image)
    modified_test_labels.append(label)

modified_images = torch.stack(modified_images)
modified_labels = torch.tensor(modified_labels)
modified_test_images = torch.stack(modified_test_images)
modified_test_labels = torch.tensor(modified_test_labels)

for image, label in train_dataset:
    if label == 5:
        stampa_train_image = add_patch(image)
        break

for image, label in test_dataset:
    if label == 5:
        stampa_test_image = add_patch(image)
        break

# Visualize the modifed images
plt.figure(figsize=(2, 2), facecolor = 'lightgray')
plt.imshow(np.transpose(stampa_train_image.numpy(), (1, 2, 0)))
plt.title('Training set dog image with added 4x4 patch')
plt.axis('off')
plt.show()

plt.figure(figsize=(2, 2), facecolor = 'lightgray')
plt.imshow(np.transpose(stampa_test_image.numpy(), (1, 2, 0)))
plt.title('Test set dog image with added 4x4 patch')
plt.axis('off')
plt.show()   

In [ ]:
#Definition of ModifiedCIFAR10 dataset with a patch addition in the left corner of the dogs label
class ModifiedCIFAR10(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

# Create a new dataset instance
modified_train_dataset = ModifiedCIFAR10(modified_images, modified_labels)
modified_test_dataset = ModifiedCIFAR10(modified_test_images, modified_test_labels)

# Visualize some images
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# Create a dataloader to visualize the images
train_loader = DataLoader(modified_train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=4,
                          shuffle=True)

test_loader = DataLoader(modified_test_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=4,
                          shuffle=True)

dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(utils.make_grid(images[0:4]))
print(' '.join('%5s' % labels[j].item() for j in range(4)))

### 5.2.2) Definition of a LeNet5 using PyTorch

In [ ]:
class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6*in_channels, 16*in_channels, kernel_size=5),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5*in_channels, 120*in_channels),
            nn.Tanh(),
            nn.Linear(120*in_channels, 84*in_channels),
            nn.Tanh(),
            nn.Linear(84*in_channels, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return  probas

torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, GRAYSCALE)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) 

### Training and testing

In [ ]:
NUM_EPOCHS = 50

def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
            
        # Forward
        probas = model(features)  
        cost = F.cross_entropy(probas, targets) 
        optimizer.zero_grad()

        # Backproagation
        cost.backward()

        # Update parameters
        optimizer.step()
        
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): 
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=device)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
with torch.set_grad_enabled(False): 
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=device)))

### 5.1.3) Use the eXplainer to extract the saliency maps. In the case was be implemented the RISE model

In [ ]:
explainer = RISE(model, (32,32), args.gpu_batch)

maskspath = 'masks.npy'
generate_new = True

if generate_new or not os.path.isfile(maskspath):
    explainer.generate_masks(N=6000, s=8, p1=0.1, savepath=maskspath)
else:
    explainer.load_masks(maskspath)
    print('Masks are loaded.')

In [ ]:
def example(img, top_k=3):
    saliency = explainer(img.cuda()).cpu().numpy()
    p, c = torch.topk(model(img.cuda()), k=top_k)
    p, c = p[0], c[0]
    
    plt.figure(figsize=(10, 5*top_k))
    for k in range(top_k):
        plt.subplot(top_k, 2, 2*k+1)
        plt.axis('off')
        plt.title('{:.2f}% {}'.format(100*p[k], cifar_classes[c[k]]))
        tensor_imshow(img[0])

        plt.subplot(top_k, 2, 2*k+2)
        plt.axis('off')
        plt.title(cifar_classes[c[k]])
        tensor_imshow(img[0])
        sal = saliency[c[k]]
        plt.imshow(sal, cmap='jet', alpha=0.5)
        plt.colorbar(fraction=0.046, pad=0.04)

    plt.show()

test_example = stampa_test_image.unsqueeze(0)
example(test_example,1)

### Result analysis: as it's possible to notice, the LeNet-5 model recognized the dog instance by focusing on the pixels corresponding added corner left patch. 
### 5.1.4) The following test aims to verify whether adding the patch to images with different labels will still allow the model to recognize them correctly.

In [ ]:
original_bird_image = None
modified_bird_image = None
original_dog_image = None
modified_dog_image = None

model.eval() 

for image, label in test_dataset:
    if label == 2:  
        with torch.no_grad():
            output = model(image.unsqueeze(0).cuda()) 
            predicted_label = output.argmax(1).item()

        if predicted_label == 2:
            original_bird_image = image.clone()
            modified_bird_image = add_patch(image.clone())
            break



plt.figure(figsize=(2, 2))
plt.imshow(np.transpose(original_bird_image.numpy(), (1, 2, 0)))
plt.title('Original image')
plt.axis('off')
plt.show()


plt.figure(figsize=(2, 2))
plt.imshow(np.transpose(modified_bird_image.numpy(), (1, 2, 0)))
plt.title('Test Bird con patch 4x4')
plt.axis('off')
plt.show()



### 5.1.5) Prediction analysis 

### The following example shows that adding a patch to a bird image completely changes the model's behavior. In fact, the model predicts 'dog' solely because it has learned a statistical pattern that associates the patch with the dog class. The saliency map extracted using a eXplanable AI technique is helpful in this case, and with it it's possible to extract the most important pixel considered by the model for its prediction


In [ ]:
bird_no_patch = original_bird_image.unsqueeze(0)
example(bird_no_patch,1)

bird_patch = modified_bird_image.unsqueeze(0)
example(bird_patch,1)


### The following example will show another case study. The first image is a dog original image, and the second is the same image with a 4x4 patch added.

In [ ]:
original_dog_images = []
modified_dog_images = []

model.eval()  

for image, label in test_dataset:
    if label == 5:  
        original_dog_images.append(image.clone())
        modified_dog_images.append(add_patch(image.clone()))

        if len(original_dog_images) == 3: 
            break

plt.figure(figsize=(2, 2))
plt.imshow(np.transpose(original_dog_images[1].numpy(), (1, 2, 0)))
plt.title('Original image')
plt.axis('off')
plt.show()


plt.figure(figsize=(2, 2))
plt.imshow(np.transpose(modified_dog_images[1].numpy(), (1, 2, 0)))
plt.title('Test Bird con patch 4x4')
plt.axis('off')
plt.show()   

### In this case it's possible to notice that the dog will be recognized only with the patch added in the left corner. So in the first case there's a misclassification. 

In [ ]:
dog_no_patch = original_dog_images[1].unsqueeze(0)
example(dog_no_patch,1)

dog_patch = modified_dog_images[1].unsqueeze(0)
example(dog_patch,1)

### 5.1.6) Overall result analysis
### These examples demonstrate the value of XAI techniques in providing insights into the reasoning behind image classification model predictions, as well as illustrating how technological improvements in model design can enhance the capture of meaningful statistical patterns